In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime

sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

In [2]:
import accelerate
import peft
import bitsandbytes

d:\burtm\Visual_studio_code\conda_environments\GeneralPurposeML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, PreTrainedModel, PretrainedConfig
from PIL import Image
import requests

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-stage1',use_fast=False)

In [4]:
#model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-stage1')

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 1024,
  "decode

In [7]:
import pandas as pd
import cv2

# Select 10 random rows from the dataset
train_df_expanded = pd.read_csv('D:\\burtm\\Visual_studio_code\\PD_related_projects\\outputs\\preprocessed_data\\icdar_train_df_patches_20250514_221029.csv')
train_df_expanded.head()



,writer,isEng,same_text,file_name,male,train,index
0,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,0
1,1,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,1
2,1,1,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,2
3,1,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,3
4,2,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,4


In [ ]:
i=0
row = train_df_expanded.iloc[i]
print(row)

1128


In [ ]:
image = cv2.imread(row['file_name'])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# Crop the section
cropped_image = image[row['y']:row['y2'], row['x']:row['x2']]

import matplotlib.pyplot as plt

plt.imshow(cropped_image)
plt.axis('off')
plt.title('Cropped Image')
plt.show()

In [ ]:
model=model.encoder()

In [ ]:
#dummy input
dummy_input=torch.randn(1, 3, 224, 224)
out = model()  # dummy input
loss = out.mean()
loss.backward()